In [1]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
llm_response = llm.invoke("Tell me a joke")
print(llm_response)

content='Why did the scarecrow win an award? \n\nBecause he was outstanding in his field!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 11, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bo7ZrQR4fFOSFhwzLbYxbt78AJDlo', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--3676b21d-e577-4d15-b278-f93cd0efe0d4-0' usage_metadata={'input_tokens': 11, 'output_tokens': 18, 'total_tokens': 29, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [2]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
chain = llm | output_parser
result = chain.invoke("Tell me a joke")
print(result)


Why don’t scientists trust atoms?

Because they make up everything!


In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
chain = prompt | llm | output_parser
result = chain.invoke({"topic": "programming"})
print(result)


Why do programmers prefer dark mode?

Because light attracts bugs!


In [4]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a helpful assistant that tells jokes."),
    HumanMessage(content="Tell me about programming")
]
response = llm.invoke(messages)
print(response)

template = ChatPromptTemplate([
    ("system", "You are a helpful assistant that tells jokes."),
    ("human", "Tell me about {topic}")
])
chain = template | llm
response = chain.invoke({"topic": "programming"})
print(response)


content="Sure, here's a fun programming joke for you:\n\nWhy do programmers prefer dark mode?\n\nBecause light attracts bugs! \n\nIf you want to know more about programming concepts or need further jokes, feel free to ask!" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 24, 'total_tokens': 66, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bo7a09TOtqGfaRDgGzhuyvqIsbSMs', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--f3a03ef0-2553-4255-9df2-c52ac28342a6-0' usage_metadata={'input_tokens': 24, 'output_tokens': 42, 'total_tokens': 66, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
co

Loading Documents

In [5]:
from langchain_community.document_loaders import PyPDFLoader
from typing import List
from langchain_core.documents import Document

def load_single_pdf(file_path: str) -> List[Document]:
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    return documents

file_path = r"D:\Project\Generative AI\Productimate.io\Content\Productimate Brochure.pdf"
documents = load_single_pdf(file_path)
print(f"Loaded {len(documents)} documents from the file.")


Loaded 6 documents from the file.


Splitting Documents

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

splits = text_splitter.split_documents(documents)
print(f"Split the documents into {len(splits)} chunks.")


Split the documents into 12 chunks.


In [7]:
print(splits[0].metadata)


{'producer': 'Skia/PDF m139 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Productimate Brochure', 'source': 'D:\\Project\\Generative AI\\Productimate.io\\Content\\Productimate Brochure.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}


In [8]:
print(documents[0])


page_content='1.  Company  Profile  
Legal  Structure:  Proprietorship  (GST-registered  May  27,  2025)  
Trade
 
Name:
 
Productimate
 
AI
 
Solutions
 
Tagline:
 
Smart
 
Tech,
 
Seamless
 
Solution
 
Headquarters:
 
Gurgaon,
 
India
 Website:  productimate.io  LinkedIn:  linkedin.com/company/productimate-ai-solutions 
 
2.  Mission,  Vision  &  Values  
●  Mission:  
Empower
 
startups
 
and
 
agencies
 
to
 
achieve
 
rapid
 
growth
 
with
 
accessible,
 
high-quality
 
technology,
 
AI
 
automation
 
and
 
design,
 
delivered
 
on
 
time
 
and
 
within
 
budget.
 ●  Vision:  
To
 
be
 
the
 
go-to
 
technology
 
partner
 
for
 
early-stage
 
ventures
 
worldwide—known
 
for
 
flexibility,
 
transparency
 
and
 
SEO-first
 
thinking.
 ●  Core  Values:  1.  Client-Centricity:  We  listen  deeply,  adapt  quickly,  deliver  precisely.  2.  Innovation:  We  embrace  new  tools—cross-platform,  serverless,  generative  
AI—to
 
solve
 
real
 
problems.
 3.  Excellence:  We  maintain  

Use OpenAi embedding

In [9]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
document_embeddings = embeddings.embed_documents([split.page_content for split in splits])
print(f"Created embeddings for {len(document_embeddings)} document chunks.")


Created embeddings for 12 document chunks.


In [10]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
document_embeddings = embedding_function.embed_documents([split.page_content for split in splits])
print(document_embeddings[0][:5])  # Printing first 5 elements of the first embedding


C:\Users\saura\AppData\Local\Temp\ipykernel_19496\2990070485.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
d:\Project\Generative AI\Productimate.io\Content\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[-0.09933753311634064, -0.051040176302194595, -0.007813992910087109, -0.06836812943220139, 0.05207698047161102]


Creating vector store

In [11]:
from langchain_chroma import Chroma

collection_name = "my_collection"
vectorstore = Chroma.from_documents(
    collection_name=collection_name,
    documents=splits,
    embedding=embedding_function,
    persist_directory="./chroma_db"
)
print("Vector store created and persisted to './chroma_db'")


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


Vector store created and persisted to './chroma_db'


In [12]:
query = "When was Productimate AI solution found"
search_results = vectorstore.similarity_search(query, k=2)
print(f"\nTop 2 most relevant chunks for the query: '{query}'\n")
for i, result in enumerate(search_results, 1):
    print(f"Result {i}:")
    print(f"Source: {result.metadata.get('source', 'Unknown')}")
    print(f"Content: {result.page_content}")
    print()


Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given



Top 2 most relevant chunks for the query: 'When was Productimate AI solution found'

Result 1:
Source: D:\Project\Generative AI\Productimate.io\Content\Productimate Brochure.pdf
Content: 1.  Company  Profile  
Legal  Structure:  Proprietorship  (GST-registered  May  27,  2025)  
Trade
 
Name:
 
Productimate
 
AI
 
Solutions
 
Tagline:
 
Smart
 
Tech,
 
Seamless
 
Solution
 
Headquarters:
 
Gurgaon,
 
India
 Website:  productimate.io  LinkedIn:  linkedin.com/company/productimate-ai-solutions 
 
2.  Mission,  Vision  &  Values  
●  Mission:  
Empower
 
startups
 
and
 
agencies
 
to
 
achieve
 
rapid
 
growth
 
with
 
accessible,
 
high-quality
 
technology,
 
AI
 
automation
 
and
 
design,
 
delivered
 
on
 
time
 
and
 
within
 
budget.
 ●  Vision:  
To
 
be
 
the
 
go-to
 
technology
 
partner
 
for
 
early-stage
 
ventures
 
worldwide—known
 
for
 
flexibility,
 
transparency
 
and
 
SEO-first
 
thinking.
 ●  Core  Values:  1.  Client-Centricity:  We  listen  deeply,  adapt  quickl

Create Retriever

In [13]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
retriever_results = retriever.invoke("what is the vision of Productimate AI Solution")
print(retriever_results)


[Document(id='ff814a05-05c9-4bdb-9f35-12ab1c9900f8', metadata={'total_pages': 6, 'title': 'Productimate Brochure', 'creationdate': '', 'page_label': '1', 'producer': 'Skia/PDF m139 Google Docs Renderer', 'page': 0, 'creator': 'PyPDF', 'source': 'D:\\Project\\Generative AI\\Productimate.io\\Content\\Productimate Brochure.pdf'}, page_content='1.  Company  Profile  \nLegal  Structure:  Proprietorship  (GST-registered  May  27,  2025)  \nTrade\n \nName:\n \nProductimate\n \nAI\n \nSolutions\n \nTagline:\n \nSmart\n \nTech,\n \nSeamless\n \nSolution\n \nHeadquarters:\n \nGurgaon,\n \nIndia\n Website:  productimate.io  LinkedIn:  linkedin.com/company/productimate-ai-solutions \n \n2.  Mission,  Vision  &  Values  \n●  Mission:  \nEmpower\n \nstartups\n \nand\n \nagencies\n \nto\n \nachieve\n \nrapid\n \ngrowth\n \nwith\n \naccessible,\n \nhigh-quality\n \ntechnology,\n \nAI\n \nautomation\n \nand\n \ndesign,\n \ndelivered\n \non\n \ntime\n \nand\n \nwithin\n \nbudget.\n ●  Vision:  \nTo\n \n

# RAG Chain

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_openai import ChatOpenAI


system_prompt = """You are an assistant for summarizing content. Use the retrieved context to highlight key points."""
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "Summarize the following content: {context}\n\nUser input: {input}")
])

llm = ChatOpenAI(model="gpt-4")

document_chain = create_stuff_documents_chain(llm, prompt)


rag_chain = create_retrieval_chain(retriever, document_chain)


result = rag_chain.invoke({"input": "Summarize the main points."})
print(result["answer"])


The content discusses various stages of a software development project, namely:

1. Initiation & Discovery: A one-week sprint takes place to create user stories, wireframes, and finalize the tech stack.
2. Agile Sprints: Work is conducted in two-week iterations, including daily meetings and bi-weekly demos.
3. CI/CD & Testing: The project involves unit, integration, and end-to-end tests, linting, coverage thresholds, and Dockerized builds which lead to staging and production deployment.
4. Code Reviews & Pair Programming: Intern-led squads review Pull Requests under CTO or Lead supervision.
5. SEO & Performance Audits: Steps are taken to ensure Search Engine Optimization and satisfactory website performance, with checks for Lighthouse scores, Core Web Vitals, and accessibility.
6. User Acceptance & Handover: Once developed, the project is finalized with detailed documentation, training sessions, and one month of support. 

There is also mention of a case study for a Live-Commerce Aucti